**Install the required packages for the code**




In [3]:
#@title <h3>Installing all the Packages, Model and Dependencies
#@markdown * Cloning the github repo.
#@markdown * Install Packages
#@markdown *Load the model
%%capture
#Cloning the Github Repo
!git clone https://github.com/kmAyush/Video-Dubbing-with-Lip-Synchronization.git

#Installing the requirement.txt file
# !pip install -r "https://drive.google.com/uc?export=download&id=1sJTj6nRvwf9YzpzzCBFykpN9xDIn36er"

!pip install gdown
!pip install accelerate -q
from ipywidgets import widgets
import os, gdown, shutil, zipfile
import locale
from google.colab import files
from IPython.display import display, Audio, HTML, display
from moviepy.editor import VideoFileClip, AudioFileClip
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from base64 import b64encode
# import moviepy.editor as mp
!pip install pydub
from pydub import AudioSegment


#DISPLAY VIDEO
def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

#Wav2Lip Model
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11
#Wav2Lip Model Loaded




#STT model load
url = "https://drive.google.com/file/d/1esCdvJ5Lsr2RisWabizX6AqNHAt8B7Kc/view?usp=sharing"
file_id = url.split('/')[-2]
print(file_id)
# Destination directory
dest = '/content/Video-Dubbing-with-Lip-Synchronization/Speech to text/'

# Download the file using the extracted file ID
gdown.download(f'https://drive.google.com/uc?id={file_id}', dest, quiet=False)


print("Execution succesfull")

In [27]:
#@title <h3>Step1: Select any video from Test Sample</h1>


# Define the directory containing your video files
video_directory = '/content/Video-Dubbing-with-Lip-Synchronization/Test_samples'

# Get a list of all video files in the directory
video_files = [f for f in os.listdir(video_directory) if f.endswith('.mp4')]

# Create a dictionary to store video paths
video_paths = {}

# Define the maximum number of buttons per row
max_buttons_per_row = 3

# Create a list to store button widgets
button_widgets = []

# Create buttons for each video file and organize them into rows
for video_file in video_files:
    video_path = os.path.join(video_directory, video_file)
    video_paths[video_file] = video_path
    video_button = widgets.Button(description=video_file[:video_file.index(".")])

    def on_button_click(b):
        global selected_video
        selected_video = video_paths[b.description + ".mp4"]
        print(f"Selected video path: {selected_video}")

    video_button.on_click(on_button_click)
    button_widgets.append(video_button)

# Organize buttons into rows
rows = [widgets.HBox(button_widgets[i:i+max_buttons_per_row]) for i in range(0, len(button_widgets), max_buttons_per_row)]
display(widgets.VBox(rows))





Selected video path: /content/Video-Dubbing-with-Lip-Synchronization/Test_samples/Oppenheimer_1.mp4


In [ ]:
#@title <h5> Run the code to see the selected video
print(selected_video)
showVideo(selected_video)

In [6]:
#@title <h3>Step2: Split the selected video into video (muted) and mono-audio file</h1>
%%capture


warnings.filterwarnings("ignore")

def extract_audio_and_muted_video(input_file):
    # Load the video file
    video_clip = VideoFileClip(input_file)

    # Separate video into muted video and audio
    muted_video = video_clip.set_audio(None)
    audio_clip = video_clip.audio

    # Extract filename without extension
    filename_no_extension = os.path.splitext(os.path.basename(input_file))[0]

    if os.path.exists('/content/Step1_output'):
        # Remove the existing folder and its contents
        os.system('rm -r /content/Step1_output')

    # Create the 'Step1_output' folder
    os.mkdir('/content/Step1_output')

    # Save the muted video with specific filename, hide progress bar
    muted_video_path = f"/content/Step1_output/muted_{filename_no_extension}.mp4"
    muted_video.write_videofile(muted_video_path, verbose=False)

    # Save the audio with specific filename, hide progress bar
    audio_path = f"/content/Step1_output/audio_{filename_no_extension}.wav"
    audio_clip.write_audiofile(audio_path, verbose=False)


    return muted_video_path, audio_path


muted_video_path , extracted_audio_path = extract_audio_and_muted_video(selected_video)
!ffmpeg -y -i $extracted_audio_path -ac 1 -ar 22500  /content/Step1_output/mono_audio.wav
# Update extracted_audio_path to the mono audio file
extracted_audio_path = '/content/Step1_output/mono_audio.wav'

In [ ]:
#@title <h5>  Run the code to listen the extracted audio
print(extracted_audio_path)
Audio(extracted_audio_path, autoplay = True)


In [24]:
#@title <h3>Step3: Generate the text from the extracted audio i.e. Speech-To-Text</h1>

%%capture
"""
Prerequisites:
tensorflow

How to use this file
1. Call speech_to_text(model_filepath, wav_filepath) function

"""

warnings.filterwarnings("ignore")

frame_length = 256
frame_step = 160

ch = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
stoi = keras.layers.StringLookup(vocabulary=ch, oov_token="")
itos = keras.layers.StringLookup(vocabulary=stoi.get_vocabulary() , oov_token="", invert=True)

# Function required by models
@keras.saving.register_keras_serializable()
def CTC_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

# Encode and decode functions
def encodeSample(wavFile):
    wav = tf.io.read_file(wavFile)
    wav, _ = tf.audio.decode_wav(wav)
    wav = tf.squeeze(wav, axis=-1)
    audio = tf.cast(wav, tf.float32)
    spectogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step)
    spectogram = tf.abs(spectogram)
    spectogram = tf.math.pow(spectogram, 0.5)
    means = tf.math.reduce_mean(spectogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectogram, 1, keepdims=True)
    spectogram = (spectogram - means) / stddevs + 1e-9
    spectogram = spectogram.numpy()
    spectogram = tf.expand_dims(spectogram, 0)
    return spectogram

def decodeSample(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length = input_len, greedy = True)[0][0]
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(itos(res)).numpy().decode("utf-8")
        output_text.append(res)

    return output_text

# Main function
def speech_to_text(model,wavFile):
    model = keras.models.load_model(model)
    spectogram = encodeSample(wavFile)
    pred = model.predict(spectogram)
    return decodeSample(pred)

# Due to trained on less variety audio sample
extracted_text = speech_to_text("/content/Video-Dubbing-with-Lip-Synchronization/Speech to text/speech2text4.keras",extracted_audio_path)




In [ ]:
#@title <h3> Step3.1
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# %%capture
# Comparision with distil-whisper v3 large model
device = "cuda:0" if torch. cuda. is_available() else "cpu"
torch_dtype = torch.float16 if torch. cuda. is_available() else torch.float32

model_id = "distil-whisper/distil-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

extracted_text2 = pipe(extracted_audio_path)['text']
clear_output()
print(extracted_text[0])
print(f"extracted text form the audio --> {extracted_text2}")

In [ ]:
#@title <h3>Step4: Lets translate the generated text from the step-3 (right now in hindi)</h1>

%cd /content/Video-Dubbing-with-Lip-Synchronization/Text Translation/

!python3 trans_text.py
#%run trans_text.py
from trans_text import generate_trans_text


translated_text = generate_trans_text(extracted_text2)
clear_output()
print(f"Translated Text --> {translated_text}")

In [ ]:
#@title <h3>Step5: Generate the audio from the translated Test</h1>#
#@markdown * Initializing the xTTS-v2 model.\
#@markdown Note:- Give y as input (if asked)
!pip install TTS gradio==4.5.0 -q
from TTS.api import TTS
import gradio as gr
from IPython.display import Audio
import numpy, pandas
os.environ["COQUI_TOS_AGREED"] = "1"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)
clear_output()
print("xtts-v2 model Loaded Successfully")

In [ ]:
#@title <h3>Step6: Select the Voice model </h1>
import os
import ipywidgets as widgets
from IPython.display import display

selected_voice = ""

# Define the folder path
folder_path = '/content/Video-Dubbing-with-Lip-Synchronization/voice_model'

# Get the list of files in the folder
file_list = os.listdir(folder_path)

# Create a dictionary to store button values (file paths)
button_values = {}

# Create buttons for each file in the folder
buttons = []
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    button = widgets.Button(description=file_name[:file_name.index(".")])
    button_values[button] = file_path  # Store button value (file path) in the dictionary
    buttons.append(button)

# Function to handle button clicks
def on_button_click(button):
    global selected_voice
    selected_voice = button.description
    print(f"You selected the voice of '{button.description}'")
    # print(f"Selected voice file path: {selected_voice}")

    # You can use selected_voice in other functions here or perform further actions

# Assign the on_click event to the buttons
for button in buttons:
    button.on_click(on_button_click)

# Display the buttons
display(*buttons)


In [30]:
def generate_speech(text, speaker_wav, language):
    # Generate speech using the provided text, speaker voice, and language
    name = speaker_wav
    path = "/content/Video-Dubbing-with-Lip-Synchronization/voice_model/"
    speaker_wav = path + speaker_wav + ".mp3"
    #print(f"speaker_wav = {speaker_wav}")
    if os.path.exists('/content/TTS_output'):
      os.system('rm -r /content/TTS_output')
    os.mkdir("/content/TTS_output")
    file_path = f"/content/TTS_output/{name}_generated.wav"
    tts.tts_to_file(text=text,
                    file_path=file_path,
                    speaker_wav=speaker_wav,
                    language=language)
    return file_path

In [ ]:
#translated_text = "मुझे हिंदू धर्मग्रंथ भगवद गीता की पंक्ति याद है"

In [31]:
import librosa
length = 9999
gen_speech = ""
for i in range(4): # Iterate two pick smallest length audio sample
  speech = generate_speech(translated_text, selected_voice, "hi")
  #display(Audio(speech, autoplay=False))
  if librosa.get_duration(path=speech) < length:
    gen_speech = speech
display(Audio(gen_speech, autoplay=False))
clear_output()

In [ ]:
#@title <h3> Step7: Generating final output
#@markdown Step-7.1: Preporocessing the muted video for the Wav2lip model </b>
from base64 import b64encode
import moviepy.editor as mp

def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()

PATH_TO_YOUR_VIDEO = muted_video_path

video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 /content/sample_data/input_vid.wav")
    PATH_TO_YOUR_VIDEO = "/content/sample_data/input_vid.wav"
    print("Video resized to 720p")
    print("Muted Video")
    showVideo(PATH_TO_YOUR_VIDEO)

else:
    print("No resizing needed")
    print("Muted Video")
    showVideo(PATH_TO_YOUR_VIDEO)
    shutil.copyfile(PATH_TO_YOUR_VIDEO, "/content/sample_data/input_vid.wav")






In [ ]:
#@title Step7.2: Preprocessing the generated audio
import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Upload'
# gen_speech
tts_output = "/content/TTS_output"


def displayAudio(audio_path):
  display(Audio(audio_path))


PATH_TO_YOUR_AUDIO = gen_speech

# Load audio with specified sampling rate
import librosa
audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

# Save audio with specified sampling rate
import soundfile as sf
sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')

clear_output()






def increase_audio_speed(input_file, output_file, speed_multiplier):
    """
    Increase the speed of an audio WAV file.

    Args:
    - input_file (str): Path to the input WAV file.
    - output_file (str): Path to save the modified audio WAV file.
    - speed_multiplier (float): Factor by which to increase the speed of the audio.

    Returns:
    - None
    """
    # Load the WAV file
    audio = AudioSegment.from_wav(input_file)
    # Increase the speed
    audio_with_increased_speed = audio.speedup(playback_speed=speed_multiplier)
    # Save the modified audio to a new WAV file
    audio_with_increased_speed.export(output_file, format="wav")

# Example usage:
input_file = '/content/sample_data/input_audio.wav'
output_file = '/content/sample_data/input_audio1.wav'
speed_multiplier = librosa.get_duration(path=gen_speech) // librosa.get_duration(path=extracted_audio_path)
if speed_multiplier >1:
  speed_multiplier = 1.4
else:
  speed_multiplier = 1

increase_audio_speed(input_file, output_file, speed_multiplier)
displayAudio(output_file)

In [ ]:
#@title Step7.3: Generating the final output after getting input from the last two steps
%cd /content/Wav2Lip

# Set up paths and variables for the output file
output_file_path = '/content/Wav2Lip/results/result_voice.mp4'

# Delete existing output file before processing, if any
if os.path.exists(output_file_path):
    os.remove(output_file_path)

pad_top =  0
pad_bottom =  10
pad_left =  0
pad_right =  0
rescaleFactor =  1
nosmooth = True

use_hd_model = True
checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'

if nosmooth == False:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid.wav" --audio "../sample_data/input_audio1.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !python inference.py --checkpoint_path $checkpoint_path --face "../sample_data/input_vid.wav" --audio "../sample_data/input_audio1.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

#Preview output video
if os.path.exists(output_file_path):
    clear_output()
    print("Final Video Preview")
    print("Download this video from", output_file_path)
    showVideo(output_file_path)

else:
    print("Processing failed. Output video not found.")




<center>$Team Members$
<center>$Akhil$
<center>$Suprit$
<center>$Ayush$





**********************